In [1]:
clear

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, regularizers, optimizers
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

# 시간정보를 활용하여 폴더 생성
import datetime 

In [4]:
#3채널 525*700 픽셀의 500개 이미지
IMG_CHANNELS = 3
IMG_ROWS = 525
IMG_COLS = 700
IMG_SIZE = (IMG_ROWS, IMG_COLS)

In [5]:
VALIDATION_SPLIT = 0.10

img_gen = tf.keras.preprocessing.image.ImageDataGenerator(validation_split= VALIDATION_SPLIT)
#img_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, validation_split=VALIDATION_SPLIT) 

train_set = img_gen.flow_from_directory(directory="/content/drive/MyDrive/dataset/Generated_Spectograms",
                                        target_size = IMG_SIZE,                                                     
                                        class_mode='categorical',
                                        subset='training',
                                        batch_size = 500)

test_set = img_gen.flow_from_directory(directory="/content/drive/MyDrive/dataset/Generated_Spectograms/",
                                        target_size = IMG_SIZE,
                                        class_mode='categorical',
                                        subset='validation',
                                        #shuffle=None,
                                        batch_size = 500)

# 이미지와 이미지에 해당하는 label
Train_X, Train_Y = next(train_set)
Test_X, Test_Y = next(test_set)

Found 450 images belonging to 5 classes.
Found 50 images belonging to 5 classes.


In [8]:
# 그래프만 추출
X_train = Train_X[:, 39:467, 81:567, :]
X_test = Test_X[:, 39:467, 81:567, :]

In [9]:
# image augmentation: random_crop
def random_crop(img_set, random_crop_width):
    height, width = img_set.shape[1], img_set.shape[2]
    dx = random_crop_width
    x = np.random.randint(0, width - dx + 1)
    return img_set[ :, :, x:(x+dx), :]

In [10]:
crop_width = 16

argumented_X_train = random_crop(X_train, crop_width)
argumented_Y_train = Train_Y

for i in range(9):
  new_set = random_crop(X_train, crop_width)
  argumented_X_train = np.concatenate((argumented_X_train, new_set),axis=0)
  argumented_Y_train = np.concatenate((argumented_Y_train,Train_Y),axis=0)

argumented_X_test = random_crop(X_test, crop_width)
argumented_Y_test = Test_Y

for i in range(9):
  new_set = random_crop(X_test, crop_width)
  argumented_X_test = np.concatenate((argumented_X_test, new_set),axis=0)
  argumented_Y_test = np.concatenate((argumented_Y_test, Test_Y),axis=0)


In [11]:
argumented_X_train = tf.image.resize(argumented_X_train, (428, 81))
argumented_X_test = tf.image.resize(argumented_X_test, (428, 81))

#X_train = np.array(X_train.astype('float32'))
#X_test = np.array(X_test.astype('float32'))
argumented_X_train = np.array(argumented_X_train).astype('float32')
argumented_X_test = np.array(argumented_X_test).astype('float32')

In [12]:
# 복수 객체 저장 및 불러오기
array2 = np.arange(10, 20)
np.savez('/content/drive/MyDrive/dataset/augmented_Spectogram/train_16.npz', array1 = argumented_X_train, array2 = argumented_Y_train)
np.savez('/content/drive/MyDrive/dataset/augmented_Spectogram/test_16.npz', array1 = argumented_X_test, array2 = argumented_Y_test)

#np.savez('/content/drive/MyDrive/augmented_Spectogram/train.npz', array1 = argumented_X_train, array2 = argumented_Y_train)

data 저장한걸로 학습 수행

In [ ]:
clear

In [5]:
data = np.load('/content/drive/MyDrive/dataset/augmented_Spectogram/train_64.npz')
argumented_X_train = data['array1']
argumented_Y_train = data['array2']

data = np.load('/content/drive/MyDrive/dataset/augmented_Spectogram/test_64.npz')
argumented_X_test = data['array1']
argumented_Y_test = data['array2']

In [6]:
print(argumented_X_test.shape)

(500, 428, 81, 3)
